# dbachecks PR check on Beard NUC - MultipleInstances

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local source control and ensure that we are in the PR branch

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
git status

On branch development
Your branch is up to date with 'origin/development'.

nothing to commit, working tree clean


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1
$null= Reset-DbcConfig

Excellent, that worked. Now to run some checks against a single instance  

I don't care if the test pass or fail, I am only interested in any 

> Describe block has Error

or

> Error in Script

type failures

Multiple Instances at parameter

In [4]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Invoke-DbcCheck -ComputerName $Sqlinstances -Check SqlBrowserServiceAccount

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1' with Tags SqlBrowserServiceAccount

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1

  Describing SQL Browser Service

    Context Testing SQL Browser Service on SQL2005Ser2003
      [-] SQL browser service on SQL2005Ser2003 should be Stopped as only one instance is installed 75ms
        Expected strings to be the same, because Unless there are multiple instances you dont need the browser service, but they were different.
        String lengths are both 7.
        Strings differ at index 0.
        Expected: 'Stopped'
        But was:  'Running'
        1184:                             $Services.Where{ $_.ServiceType -eq 'Browser' }.State | Should -Be "Stopped" -Because 'Unless there are multiple inst

Set the config and run again

In [8]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Set-DbcConfig -Name skip.security.sadisabled -Value $false
Invoke-DbcCheck -SqlInstance $Sqlinstances -Check SaDisabled


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1' with Tags SaDisabled

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2005Ser2003
      [+] sa login is disabled on SQL2005Ser2003 50ms

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2016N2
      [+] sa login is disabled on SQL2016N2 2ms

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2012Ser08AG1
      [+] sa login is disabled on SQL2012Ser08AG1 2ms

  Describing SA Login Disabled

    Context Checking that sa login has been disabled on SQL2016N3
      [+] sa login is disabled on SQL2016N3 3ms

  Describing SA Login Disable

In [11]:
Get-DbaServerRole -SQLInstance sql2019n5 -ServerRole "sysadmin"


ComputerName : SQL2019N5
InstanceName : MSSQLSERVER
SqlInstance  : SQL2019N5
Role         : sysadmin
Login        : {sa, THEBEARD\Rob, NT SERVICE\SQLWriter, NT SERVICE\Winmgmt…}
Owner        : sa
IsFixedRole  : True
DateCreated  : 13/04/2009 12:59:06
DateModified : 13/04/2009 12:59:06




In [9]:
@(Get-DbaLogin -SQLInstance sql2019n5 -Type SQL | Where-Object { $_.PasswordPolicyEnforced -eq $false -and $_.IsDisabled -eq $false}).Count

1


In [10]:
@(Get-DbaLogin -SqlInstance sql2019n5 -Login sa).Count

1
